## Assignment: RNN and ConvNets (60 points)

The data file ”data.csv” contains 3 time series x1, x2, and y along with the
corresponding date column. The data ranges from beginning of 2019 to the end
of Feb. of 2020. The objective of this problem is to make predictions for y for
March 1st and 2nd in 2020.

Kai Hsin Hung | Harshitha Mallappa

### 1. Explore regular feedforward neural network models for this problem. (10points)

In [23]:
import pandas as pd
import numpy as np

In [24]:
data = pd.read_csv('timeseriesData.csv')
data.head()

,Date,x1,x2,y
0,1/1/19,51.0,5.550000,65.58
1,1/2/19,51.0,8.950000,65.35
2,1/3/19,43.0,7.033333,69.80
3,1/4/19,43.0,7.033333,69.76
4,1/5/19,53.0,4.950000,70.48


In [25]:
data.info()
missing_val = data.isna().sum()
print(f'Missing val:\n{missing_val}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    425 non-null    object 
 1   x1      387 non-null    float64
 2   x2      387 non-null    float64
 3   y       387 non-null    float64
dtypes: float64(3), object(1)
memory usage: 13.5+ KB
Missing val:
Date     2
x1      40
x2      40
y       40
dtype: int64


In [26]:
# data imputation for Na values
for col in ['x1', 'x2', 'y', 'Date']:
    data[col] = data[col].ffill()
missing_val = data.isna().sum()
print(f'Missing val:\n{missing_val}')

# Change Date dtype ot Date and sort it
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

feature = ['x1', 'x2']
target = 'y'

Missing val:
Date    0
x1      0
x2      0
y       0
dtype: int64


/var/folders/f8/_j25ckdn20vbvtb_vyn8bz0r0000gn/T/ipykernel_36204/2962500903.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'])


In [27]:
data

,Date,x1,x2,y
0,2019-01-01,51.0,5.550000,65.58
1,2019-01-02,51.0,8.950000,65.35
2,2019-01-03,43.0,7.033333,69.80
3,2019-01-04,43.0,7.033333,69.76
4,2019-01-05,53.0,4.950000,70.48
...,...,...,...,...
422,2020-02-27,19.0,5.483333,83.62
423,2020-02-28,19.0,5.483333,83.62
424,2020-02-29,65.0,6.183333,68.53
425,2020-02-29,65.0,6.183333,68.53


In [28]:
# split the data set, portion: 70% of training  15% of validation 15% of tes
X = data[feature].values
y = data[target].values
train_portion = round(X.shape[0] * 0.7)
val_portion = round(X.shape[0] * 0.15)
train_data_y = y[:train_portion].reshape(-1, 1)
val_data_y = y[train_portion:train_portion+val_portion].reshape(-1, 1)
test_data_y = y[train_portion+val_portion:].reshape(-1, 1)
# ensure the portion of dataset
print(f"training: {len(train_data_y)}, val: {len(val_data_y)}, test: {len(test_data_y)}")


training: 299, val: 64, test: 64


In [29]:
# preprocessing
from sklearn.preprocessing import MinMaxScaler


sc = MinMaxScaler(feature_range=(0, 1))
sc.fit(train_data_y)
train_norm = sc.transform(train_data_y)
val_norm = sc.transform(val_data_y)
test_norm = sc.transform(test_data_y)

In [30]:
# convert sequence function
def to_sequence(data, look_back, foresight):
    X, Y = [], []
    for i in range(len(data) - (look_back + foresight)):
        look_back_seq = data[i:(i+look_back), 0]
        foresight_seq = (data[i + (look_back+foresight), 0])
        X.append(look_back_seq)
        Y.append(foresight_seq)
    return np.array(X), np.array(Y)

In [31]:
# setting look back and foresight like the lecture
train_seqX, train_seqY = to_sequence(train_norm, look_back=7, foresight=6)
val_seqX, val_seqY = to_sequence(val_norm, look_back=7, foresight=6)
test_seqX, test_seqY = to_sequence(test_norm, look_back=7, foresight=6)

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Input
feedforward_model = Sequential()
# set input_shape = 7 to align look_back number
# Dense model only have 2D input
feedforward_model.add(Input(shape = (7, )))
feedforward_model.add(Dense(64, activation='relu'))
feedforward_model.add(Dense(32, activation='relu'))
feedforward_model.add(Dense(1, activation='linear'))
feedforward_model.compile(loss='mae', optimizer='adam', metrics=['mean_absolute_error'])
feedforward_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,625 (10.25 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
from keras.callbacks import EarlyStopping
checkpoint = EarlyStopping(monitor='val_loss', patience=5, mode='auto',  restore_best_weights = True)
callback_list = [checkpoint]
feedforward_network = feedforward_model.fit(train_seqX, train_seqY, validation_data=(val_seqX, val_seqY),
                                            epochs = 100, batch_size = 64, callbacks = callback_list)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0477 - mean_absolute_error: 0.0477 - val_loss: 0.0121 - val_mean_absolute_error: 0.0121
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0205 - mean_absolute_error: 0.0205 - val_loss: 0.0175 - val_mean_absolute_error: 0.0175
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0213 - mean_absolute_error: 0.0213 - val_loss: 0.0131 - val_mean_absolute_error: 0.0131
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0184 - mean_absolute_error: 0.0184 - val_loss: 0.0142 - val_mean_absolute_error: 0.0142
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0137 - mean_absolute_error: 0.0137 - val_loss: 0.0130 - val_mean_absolute_error: 0.0130
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0117 - mean_absolute_error: 0.0117 - val_loss: 0.0132 - val_mean_absolute_error: 0.0132


In [34]:
from sklearn.metrics import mean_absolute_error
test_norm_predict = feedforward_model.predict(test_seqX)
# convert back to unnormalize
test_predict = sc.inverse_transform(test_norm_predict)
testY = sc.inverse_transform(test_seqY.reshape(-1, 1))
test_Mae = mean_absolute_error(testY, test_predict)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


#### (a) Report the unnormalized MAE of the test set on your best model.

#### (b) Plot the loss curves for training and validation sets for the best model.

In [35]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y = feedforward_network.history['loss'], mode = 'lines', name='Training error'))
fig.add_trace(go.Scatter(y = feedforward_network.history['val_loss'], mode='lines', name='Validation error'))
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'Mean Absolute error', title_text=f'Unnormalized MAE = {test_Mae:.3f}')
fig.show()

#### (c) What are the predicted values of y for March 1st and March 2nd?

In [36]:
# get last 7 data due to look back setting to 7
data_norm = sc.transform(y.reshape(-1, 1))
last_7_values = data_norm[-7:].reshape(1, 7)
pred_march1_norm = feedforward_model.predict(last_7_values)
pred_march1_feedforward = sc.inverse_transform(pred_march1_norm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [37]:
last_pred_combine = np.concatenate([data_norm.flatten(), pred_march1_norm.flatten()])
updated_last_7_values = last_pred_combine[-7:].reshape(1, 7)
pred_march2_norm = feedforward_model.predict(updated_last_7_values)
pred_march2_feedforward = sc.inverse_transform(pred_march2_norm)
print(f"Feed forward model prediction for March 1st {pred_march1_feedforward[0][0]:.3f}, prediction for March 2nd {pred_march2_feedforward[0][0]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Feed forward model prediction for March 1st 70.340, prediction for March 2nd 73.539


### 2. Explore recurrent neural network models for this problem. (10 points)

#### LSTM

In [38]:
from keras.layers import LSTM
LSTM_model = Sequential()
LSTM_model.add(Input(shape=(7, 1)))
LSTM_model.add(LSTM(32, dropout = 0.1, recurrent_dropout = 0.1))
LSTM_model.add(Dense(1, activation='linear'))
LSTM_model.compile(loss='mae', optimizer='adam', metrics=['mean_absolute_error'])
LSTM_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
checkpoint = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
callback_list = [checkpoint]
LSTM_network = LSTM_model.fit(train_seqX, train_seqY, validation_data=(val_seqX, val_seqY), 
                              epochs = 100, batch_size = 64, callbacks = callback_list)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0304 - mean_absolute_error: 0.0304 - val_loss: 0.0121 - val_mean_absolute_error: 0.0121
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0151 - mean_absolute_error: 0.0151 - val_loss: 0.0143 - val_mean_absolute_error: 0.0143
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0167 - mean_absolute_error: 0.0167 - val_loss: 0.0120 - val_mean_absolute_error: 0.0120
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0138 - mean_absolute_error: 0.0138 - val_loss: 0.0125 - val_mean_absolute_error: 0.0125
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0183 - mean_absolute_error: 0.0183 - val_loss: 0.0119 - val_mean_absolute_error: 0.0119
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0127 - mean_absolute_error: 0.0127 - val_loss: 0.0119 - val_mean_absolute_error: 0.0119
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0150 - mean_absolute_error: 0.0150 - val_loss: 0.0118 - va

In [40]:
test_norm_predict = LSTM_model.predict(test_seqX)
test_predict = sc.inverse_transform(test_norm_predict)
testY = sc.inverse_transform(test_seqY.reshape(-1, 1))
testLSTM_Mae = mean_absolute_error(testY, test_predict)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


#### (a) Report the unnormalized MAE of the test set on your best model.

#### (b) Plot the loss curves for training and validation sets for the best model.

In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=LSTM_network.history['loss'], mode='lines', name='Training error'))
fig.add_trace(go.Scatter(y=LSTM_network.history['val_loss'], mode='lines', name='Validation error'))
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'Mean absolute error', title_text = f'Unnormalized MAE = {testLSTM_Mae:.3f}')
fig.show()

#### (c) What are the predicted values of y for March 1st and March 2nd?

In [42]:
data_norm = sc.transform(y.reshape(-1, 1))
last_7_values = data_norm[-7:].reshape(1, 7)
pred_march1_norm = LSTM_model.predict(last_7_values)
pred_march1_LSTM = sc.inverse_transform(pred_march1_norm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [43]:
last_pred_combine = np.concatenate([data_norm.flatten(), pred_march1_norm.flatten()])
updated_last_7_values = last_pred_combine[-7:].reshape(1, 7)
pred_march2_norm = LSTM_model.predict(updated_last_7_values)
pred_march2_LSTM = sc.inverse_transform(pred_march2_norm)
print(f"LSTM model prediction for March 1st {pred_march1_LSTM[0][0]:.3f}, prediction for March 2nd {pred_march2_LSTM[0][0]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
LSTM model prediction for March 1st 72.894, prediction for March 2nd 72.851


#### GRU

In [44]:
from keras.layers import GRU
GRU_model = Sequential()
GRU_model.add(Input(shape=(7, 1)))
GRU_model.add(GRU(32, dropout = 0.1, recurrent_dropout = 0.1))
GRU_model.add(Dense(1, activation='linear'))
GRU_model.compile(loss='mae', optimizer='adam', metrics=['mean_absolute_error'])

In [45]:
checkpoint = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
callback_list = [checkpoint]
GRU_model_network = GRU_model.fit(train_seqX, train_seqY, validation_data=(val_seqX, val_seqY), 
                                  epochs = 100, batch_size = 64, callbacks = callback_list)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0275 - mean_absolute_error: 0.0275 - val_loss: 0.0177 - val_mean_absolute_error: 0.0177
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0203 - mean_absolute_error: 0.0203 - val_loss: 0.0124 - val_mean_absolute_error: 0.0124
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0185 - mean_absolute_error: 0.0185 - val_loss: 0.0152 - val_mean_absolute_error: 0.0152
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0156 - mean_absolute_error: 0.0156 - val_loss: 0.0119 - val_mean_absolute_error: 0.0119
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0144 - mean_absolute_error: 0.0144 - val_loss: 0.0131 - val_mean_absolute_error: 0.0131
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0160 - mean_absolute_error: 0.0160 - val_loss: 0.0119 - val_mean_absolute_error: 0.0119
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0153 - mean_absolute_error: 0.0153 - val_loss: 0.0120 - va

In [46]:
test_norm_predict = GRU_model.predict(test_seqX)
print(test_norm_predict.shape)
test_predict = sc.inverse_transform(test_norm_predict)
testY = sc.inverse_transform(test_seqY.reshape(-1, 1))
test_GRUMae = mean_absolute_error(testY, test_predict)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/stepWARNING:tensorflow:6 out of the last 10 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x17c6053a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
(51, 1)


#### (a) Report the unnormalized MAE of the test set on your best model.

#### (b) Plot the loss curves for training and validation sets for the best model.

In [47]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=GRU_model_network.history['loss'], mode='lines', name='Traning error'))
fig.add_trace(go.Scatter(y=GRU_model_network.history['val_loss'], mode='lines', name='Validation error'))
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'Mean absolute error', title_text = f'Unnormalized MAE: {test_GRUMae:.3f}')

#### (c) What are the predicted values of y for March 1st and March 2nd?

In [48]:
data_norm = sc.transform(y.reshape(-1, 1))
last_7_values = data_norm[-7:]
pred_march1_norm = GRU_model.predict(last_7_values)
pred_march1_GRU = sc.inverse_transform(pred_march1_norm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


In [49]:
last_pred_combine = np.concatenate([data_norm.flatten(), pred_march1_norm.flatten()])
updated_last_7_values = last_pred_combine[-7:].reshape(1, 7)
pred_march2_norm = GRU_model.predict(updated_last_7_values)
pred_march2_GRU = sc.inverse_transform(pred_march2_norm)
print(f"Prediction for March 1st {pred_march1_GRU[0][0]:.3f}, prediction for March 2nd {pred_march2_GRU[0][0]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Prediction for March 1st 67.025, prediction for March 2nd 72.495


In [50]:
print('Compare LSTM and GRU')
print(f'LSTM:\nMAE: {testLSTM_Mae:.3f}\nMarch 1st: {pred_march1_LSTM[0][0]:.3f} March 2nd: {pred_march2_LSTM[0][0]:.3f}')
print(f'GRU:\nMAE: {test_GRUMae:.3f}\nMarch 1st: {pred_march1_GRU[0][0]:.3f} March 2nd: {pred_march2_GRU[0][0]:.3f}')

Compare LSTM and GRU
LSTM:
MAE: 6.490
March 1st: 72.894 March 2nd: 72.851
GRU:
MAE: 6.484
March 1st: 67.025 March 2nd: 72.495


### 3. Explore 1d convolutional neural network models for this problem. (10 points)

In [51]:
from keras.layers import Conv1D,MaxPool1D, Flatten

conv_model = Sequential()
# kernel size = the step of model going throuhg
# pool size = taking max of the number
conv_model.add(Input(shape=(7, 1)))
conv_model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
conv_model.add(MaxPool1D(pool_size=2))
conv_model.add(Flatten())
conv_model.add(Dense(64, activation='relu'))
conv_model.add(Dense(1, activation='linear'))
conv_model.compile(loss='mae', optimizer='adam', metrics=['mean_absolute_error'])

In [52]:
checkpoint = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
callback_list = [checkpoint]
conv_model_network = conv_model.fit(train_seqX, train_seqY, validation_data=(val_seqX, val_seqY), 
                                  epochs = 100, batch_size = 64, callbacks = callback_list)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0248 - mean_absolute_error: 0.0248 - val_loss: 0.0176 - val_mean_absolute_error: 0.0176
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0188 - mean_absolute_error: 0.0188 - val_loss: 0.0125 - val_mean_absolute_error: 0.0125
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0132 - mean_absolute_error: 0.0132 - val_loss: 0.0125 - val_mean_absolute_error: 0.0125
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0177 - mean_absolute_error: 0.0177 - val_loss: 0.0128 - val_mean_absolute_error: 0.0128
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0152 - mean_absolute_error: 0.0152 - val_loss: 0.0121 - val_mean_absolute_error: 0.0121
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0124 - mean_absolute_error: 0.0124 - val_loss: 0.0122 - val_mean_absolute_error: 0.0122
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0168 - mean_absolute_error: 0.0168 - val_loss: 0.0123 - va

In [53]:
test_norm_predict = conv_model.predict(test_seqX)
test_predict = sc.inverse_transform(test_norm_predict)
testconv_Mae = mean_absolute_error(testY, test_predict)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


#### (a) Report the unnormalized MAE of the test set on your best model.

#### (b) Plot the loss curves for training and validation sets for the best model.

In [54]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=conv_model_network.history['loss'], mode='lines', name='Training error'))
fig.add_trace(go.Scatter(y=conv_model_network.history['val_loss'], mode='lines', name='Validation error'))
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'Mean absolute error', title_text = f'Unnormalized MAE: {testconv_Mae:.3f}')

#### (c) What are the predicted values of y for March 1st and March 2nd?

In [55]:
data_norm = sc.transform(y.reshape(-1, 1))
# for align with conv model
last_7_values = data_norm[-7:].reshape(1, 7)
pred_march1_norm = conv_model.predict(last_7_values)
pred_march1_conv = sc.inverse_transform(pred_march1_norm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [56]:
last_pred_combine = np.concatenate([data_norm.flatten(), pred_march1_norm.flatten()])
updated_last_7_values = last_pred_combine[-7:].reshape(1, 7)
pred_march2_norm = GRU_model.predict(updated_last_7_values)
pred_march2_conv  = sc.inverse_transform(pred_march2_norm)
print(f"Prediction for March 1st {pred_march1_GRU[0][0]:.3f}, prediction for March 2nd {pred_march2_conv[0][0]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Prediction for March 1st 67.025, prediction for March 2nd 72.795


### Conclusion

In [57]:
print('Feed forward:')
print(f'MAE: {test_Mae:.3f}\nMarch 1st: {pred_march1_feedforward[0][0]:.3f} March 2nd: {pred_march2_feedforward[0][0]:.3f}\n')
print('Recurrent network:')
print(f'LSTM:\nMAE: {testLSTM_Mae:.3f}\nMarch 1st: {pred_march1_LSTM[0][0]:.3f} March 2nd: {pred_march2_LSTM[0][0]:.3f}\n')
print(f'GRU:\nMAE: {test_GRUMae:.3f}\nMarch 1st: {pred_march1_GRU[0][0]:.3f} March 2nd: {pred_march2_GRU[0][0]:.3f}\n')
print('Convnet 1D:')
print(f'MAE: {testconv_Mae:.3f}\nMarch 1st: {pred_march1_conv[0][0]:.3f} March 2nd: {pred_march2_conv[0][0]:.3f}')

Feed forward:
MAE: 7.014
March 1st: 70.340 March 2nd: 73.539

Recurrent network:
LSTM:
MAE: 6.490
March 1st: 72.894 March 2nd: 72.851

GRU:
MAE: 6.484
March 1st: 67.025 March 2nd: 72.495

Convnet 1D:
MAE: 7.659
March 1st: 72.505 March 2nd: 72.795


Based on those model's result, we can see the LSTM has minimum test error compare the other models